# 1st Proj

In [1]:
# pip install gradio

import torch
import torchvision
import pandas as pd
import torch.nn as nn
import numpy as np
from PIL import Image
from torchvision import transforms, datasets
import gradio as gr
target_names = ['MEL','NV','BCC','AKIEC','BKL','DF','VASC']
def prediction(input_img):
    image_pil = Image.fromarray(input_img).convert("RGB")
    model1 = torchvision.models.resnet50(weights='ResNet50_Weights.IMAGENET1K_V1')#.to(device)

    model1.fc = nn.Linear(2048, 7)#.to(device)

    # load the last checkpoint with the best model
    model1.load_state_dict(torch.load('skinmodel50.pt',map_location=torch.device('cpu'))) 

    unseen_transforms = transforms.Compose([
        transforms.Resize((224, 280)),
        transforms.CenterCrop((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

#     image = Image.open(image_pil).convert("RGB")
    image = unseen_transforms(image_pil)

    # Reshape the image to add batch dimension
    image = image.unsqueeze(0) 

    with torch.no_grad():
        model1.eval()
        inputs = image#.to(device)
        outputs = model1(inputs)
        print('outputs',outputs)
#         _, preds = torch.max(outputs, 1)
        pred = nn.functional.softmax(outputs, dim=1)#.cpu()
        print('prob',pred)
        _, pred = torch.max(pred, 1)
        print('pred',pred)
        return target_names[pred[0].item()]   #target_names[preds[0].item()],outputs
    
demo = gr.Interface(prediction, gr.Image(), "text")
if __name__ == "__main__":
    demo.launch()

C:\Users\avitr\anaconda3\envs\test_env\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


outputs tensor([[ -1.0863,   7.8576, -10.7298,  -9.0916,  -2.6903,  -7.2573, -12.7887]])
prob tensor([[1.3051e-04, 9.9984e-01, 8.4630e-09, 4.3552e-08, 2.6244e-05, 2.7265e-07,
         1.0799e-09]])
pred tensor([1])


# 2nd proj

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader, random_split
# from torch.cuda.amp import autocast, GradScaler
from torch.optim.lr_scheduler import ReduceLROnPlateau
# from torchmetrics.classification import BinaryAccuracy


# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Metrics
from sklearn.metrics import confusion_matrix

# Suppressing warnings
from warnings import filterwarnings
filterwarnings('ignore')



model1 = models.efficientnet_b7(weights = 'DEFAULT')
model1.classifier[1] = nn.Linear(model1.classifier[1].in_features, 1)
model1.load_state_dict(torch.load(r'D:\OneDrive - NITT\Custom_Download\best_model.pth',map_location=torch.device('cpu'))) 

# model2 = torch.load('/kaggle/working/best_model.pth',map_location=torch.device('cpu'))  # Loading best model of this fold
from torchvision import transforms
from PIL import Image

image_path = r"D:\OneDrive - NITT\CODE\Python Scripts\Code_2024\Zac\MultiClass\data\HAM10000_images_part_1\ISIC_0024334.jpg"


imgSize = 170
# Validation transformer
valTransformer = transforms.Compose([
    transforms.Resize(size = (imgSize, imgSize), antialias = True),
    transforms.CenterCrop(size = (imgSize, imgSize)),
    
    transforms.ToTensor(),
    transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])
])

image = Image.open(image_path).convert("RGB")
image = valTransformer(image)

# Reshape the image to add batch dimension
image = image.unsqueeze(0) 


target = ['Benign','Malignant']
with torch.no_grad():
    model1.eval()
    inputs = image.to('cpu')
    outputs = model1(inputs)
    predictions = (torch.sigmoid(outputs) > 0.5).float()
    print(target[int(predictions.item())])

C:\Users\avitr\anaconda3\envs\test_env\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


Benign


In [ ]:
import gradio as gr

# Code for Task 1
def task1(input_text):
    # Task 1 logic
    return "Task 1 Result: " + input_text

# Code for Task 2
def task2(input_image):
    # Task 2 logic
    return "Task 2 Result"

# interface one
iface1 = gr.Interface(
    fn=task1,
    inputs="text",
    outputs="text",
    title="Multi-Page Interface"
)
# interface two
iface2 = gr.Interface(
    fn=task2,
    inputs="image",
    outputs= "text",
    title="Multi-Page Interface"
)

demo = gr.TabbedInterface([iface1, iface2], ["Text-to-text", "image-to-text"])

# Run the interface
demo.launch(share=True)